# Working with many files

In `1-singlefile.ipynb` we learned how to extract subsets and reproject a single image using a variety of tools (GDAL, rasterio, xarray, rioxarray, and holoviz). Often you want to work with a whole stack of imagery - for example let's see how to create a timeseries of backscatter over [Jakobshavn_Glacier](https://en.wikipedia.org/wiki/Jakobshavn_Glacier).

GDAL VRT (Virtual Dataset) files are useful to construct mosaics or sets of multiband imagery https://gdal.org/programs/gdalbuildvrt.html. We first need to take our file list and append GDAL's `/vsicurl/` prefix to all the file names

In [1]:
import os
import xarray as xr
import rasterio

import geopandas as gpd
import hvplot.pandas
import hvplot.xarray

In [2]:
!head -n 2 gamma0.txt

https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.07.27/GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1
https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.08.02/GL_S1bks_mosaic_02Aug20_07Aug20_gamma0_50m_v03.1


In [3]:
output = 'vrt-list.txt'

with open('gamma0.txt', 'r') as f:
    gammas = f.readlines()
    vsis = ['/vsicurl/' + line for line in gammas]
    files = [os.path.basename(line) for line in gammas]
    print(f'Saving {len(vsis)} URLs to file list...')
    with open(output, 'w') as v:
        v.writelines(vsis)

!head vrt-list.txt

Saving 11 URLs to file list...
/vsicurl/https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.07.27/GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1
/vsicurl/https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.08.02/GL_S1bks_mosaic_02Aug20_07Aug20_gamma0_50m_v03.1
/vsicurl/https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.08.08/GL_S1bks_mosaic_08Aug20_13Aug20_gamma0_50m_v03.1
/vsicurl/https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.08.14/GL_S1bks_mosaic_14Aug20_19Aug20_gamma0_50m_v03.1
/vsicurl/https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.08.20/GL_S1bks_mosaic_20Aug20_25Aug20_gamma0_50m_v03.1
/vsicurl/https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.08.26/GL_S1bks_mosaic_26Aug20_31Aug20_gamma0_50m_v03.1
/vsicurl/https://n5eil11u.ecs.nsidc.org/TS1/DP0/MEASURES/NSIDC-0723.199/2020.09.01/GL_S1bks_mosaic_01Sep20_06Sep20_gamma0_50m_v03.1
/vsicurl/https://n5eil11u.ecs.nsidc.org/TS1/D

In [4]:
# Step1, create a GDAL 'VRT' file that lists everything we'd like to work with:
env_vars = 'GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR GDAL_HTTP_COOKIEFILE=.urs_cookies GDAL_HTTP_COOKIEJAR=.urs_cookies'
cog = gammas[1]
cmd = f'{env_vars} gdalbuildvrt -overwrite -allow_projection_difference -separate -input_file_list {output} stack_gamma0.vrt '
print(cmd)

GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR GDAL_HTTP_COOKIEFILE=.urs_cookies GDAL_HTTP_COOKIEJAR=.urs_cookies gdalbuildvrt -overwrite -allow_projection_difference -separate -input_file_list vrt-list.txt stack_gamma0.vrt 


In [5]:
%%time 
# Can take a bit of time b/c file CRS is checked and reprojected if necessary
!{cmd}

0...10...20...30...40...50...60...70...80...90...100 - done.
CPU times: user 26.7 ms, sys: 18.5 ms, total: 45.2 ms
Wall time: 1.24 s


In [6]:
# it's helpful to add the filename to the description for each band. you can do this with rasterio


# An alternative to using rasterio.Env() is to set global environment variables:
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR'
os.environ['GDAL_HTTP_COOKIEFILE']='.urs_cookies' 
os.environ['GDAL_HTTP_COOKIEJAR']='.urs_cookies'


with rasterio.open('stack_gamma0.vrt', 'r+') as src:
    print(src.profile)
    src.descriptions = files

{'driver': 'VRT', 'dtype': 'float32', 'nodata': -30.0, 'width': 29520, 'height': 53220, 'count': 11, 'crs': CRS.from_epsg(3413), 'transform': Affine(50.0, 0.0, -626000.0,
       0.0, -50.0, -695000.0), 'blockxsize': 128, 'blockysize': 128, 'tiled': True}


In [7]:
# Open the VRT with xarray (this is very fast b/c only metadata is read!)

da = xr.open_rasterio('stack_gamma0.vrt', chunks=dict(band=1, x=29520, y=512)) #ensure data loaded as dask arrays
da.data

,Array,Chunk
Bytes,69.13 GB,60.46 MB
Shape,"(11, 53220, 29520)","(1, 512, 29520)"
Count,1145 Tasks,1144 Chunks
Type,float32,numpy.ndarray


In [8]:
print(f'Total uncompressed dataset size= {da.nbytes/1e12} TB')

Total uncompressed dataset size= 0.0691263936 TB


In [9]:
# Assign a time coordinate instead of integer band
#import pandas as pd
#dates = [url.split('/')[-2] for url in gammas]
#print(dates[:2])

#datetimes = [pd.to_datetime(x) for x in dates]
#datetimes[:2]

In [10]:
# make 'time' active coordinate instead of integer band
#da = da.assign_coords(time=('band', datetimes))
#da = da.swap_dims({'band':'time'})
#da.name = 'gamma0'
#da

In [11]:
# Bounding box of interest 
# draw on here: http://geojson.io/

gf = gpd.read_file('jakobshavn.geojson')
bbox = gf.hvplot.polygons(alpha=0.2, geo=True, tiles=True)
bbox

:Overlay
   .Tiles.I    :Tiles   [x,y]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [12]:
# convert our bounding box to epsg:3413 (south polar sterographic)
# NOTE: miny and maxy are switched here...
gf3413 = gf.to_crs(3413)
gf3413.bounds

,minx,miny,maxx,maxy
0,-256677.224952,-2.314901e+06,-88385.054733,-2.224398e+06


In [13]:
xmin, ymax, xmax, ymin = gf3413.bounds.values[0]

In [14]:
# Now extract that subset from all the GIMP data

subset = da.sel(x=slice(xmin, xmax), y=slice(ymin, ymax))
subset.data

dask.array<getitem, shape=(11, 1810, 3366), dtype=float32, chunksize=(1, 512, 3366), chunktype=numpy.ndarray>

In [15]:
print(f'Subset uncompressed dataset size= {subset.nbytes/1e9} GB')

Subset uncompressed dataset size= 0.26806824 GB


In [16]:
%%time 
# We can easily hold ~100MB of data in memory, so let's do that to make plotting faster

subset.persist() 

CPU times: user 3.97 s, sys: 1.1 s, total: 5.08 s
Wall time: 1min 42s


<xarray.DataArray (band: 11, y: 1810, x: 3366)>
dask.array<getitem, shape=(11, 1810, 3366), dtype=float32, chunksize=(1, 512, 3366), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1 2 3 4 5 6 7 8 9 10 11
  * y        (y) float64 -2.224e+06 -2.224e+06 ... -2.315e+06 -2.315e+06
  * x        (x) float64 -2.567e+05 -2.566e+05 ... -8.848e+04 -8.842e+04
Attributes:
    transform:     (50.0, 0.0, -626000.0, 0.0, -50.0, -695000.0)
    crs:           +init=epsg:3413
    res:           (50.0, 50.0)
    is_tiled:      1
    nodatavals:    (-30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -...
    scales:        (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)
    offsets:       (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    descriptions:  ('GL_S1bks_mosaic_27Jul20_01Aug20_gamma0_50m_v03.1\n', 'GL...

In [17]:
subset.hvplot.image(rasterize=True, dynamic=True, frame_width=400, aspect='equal', cmap='gray')

:DynamicMap   [band]
   :Image   [x,y]   (value)

In [18]:
# Save this subset datacube as a local netcdf
# Xarray works best if we save a 'Dataset' rather than a 'DataArray'
ds = subset.to_dataset(name='gamma0')
ds.to_netcdf('jakobshavn.nc')

In [19]:
# round trip:
ds = xr.open_dataset('jakobshavn.nc')
ds

<xarray.Dataset>
Dimensions:  (band: 11, x: 3366, y: 1810)
Coordinates:
  * band     (band) int64 1 2 3 4 5 6 7 8 9 10 11
  * y        (y) float64 -2.224e+06 -2.224e+06 ... -2.315e+06 -2.315e+06
  * x        (x) float64 -2.567e+05 -2.566e+05 ... -8.848e+04 -8.842e+04
Data variables:
    gamma0   (band, y, x) float32 ...